# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.


In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [8]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

from google import genai

In [9]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
# Connect to OpenAI, Anthropic
openai = OpenAI()

In [30]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether
gemini = genai.Client(api_key=google_api_key)

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:

- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.


In [13]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [14]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

In [15]:
# GPT-4o-mini

completion = openai.chat.completions.create(model="gpt-4o-mini", messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she just couldn’t handle his standard deviation!


In [16]:
# GPT-4.1-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model="gpt-4.1-mini", messages=prompts, temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because they couldn't find any significant correlation!


In [17]:
# GPT-4.1-nano - extremely fast and cheap

completion = openai.chat.completions.create(model="gpt-4.1-nano", messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist go broke?

Because she kept stacking her files!


In [ ]:
# GPT-4.1
completion = openai.chat.completions.create(
    model="gpt-4.1", messages=prompts, temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the logistic regression model?

Because it just couldn’t commit!


In [19]:
# If you have access to this, here is the reasoning model o4-mini
# This is trained to think through its response before replying
# So it will take longer but the answer should be more reasoned - not that this helps..

completion = openai.chat.completions.create(model="o4-mini", messages=prompts)
print(completion.choices[0].message.content)

How many data scientists does it take to change a lightbulb?  
None—they’d rather run an A/B test to see if users actually prefer light over darkness!


In [33]:
response = gemini.models.generate_content(
    model="gemini-2.5-flash",
    contents=system_message + "\n Explain how AI works in a few words.",
)

print(response.text)

Okay, you got it! Here's one for ya:

Why did the AI break up with the calculator?
Because she said he was too "add-ictive"!

And speaking of smart decisions, how AI works in a few words:
**It's computers learning from data to spot patterns and make smart decisions.**


In [26]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google released endpoints that means you can use Gemini via the client libraries for OpenAI!
# We're also trying Gemini's latest reasoning/thinking model

gemini_via_openai_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash", messages=prompts
)
print(response.choices[0].message.content)

Why don't data scientists like to play hide-and-seek?

...Because good luck finding anything after you've dropped all the null values!


## Back to OpenAI with a serious question


In [34]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {
        "role": "system",
        "content": "You are a helpful assistant that responds in Markdown",
    },
    {
        "role": "user",
        "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown.",
    },
]

In [35]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model="gpt-4.1-mini", messages=prompts, temperature=0.7, stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ""
    reply = reply.replace("```", "").replace("markdown", "")
    update_display(Markdown(reply), display_id=display_handle.display_id)


# How to Decide if a Business Problem is Suitable for an LLM Solution

Large Language Models (LLMs) like GPT-4 are powerful tools, but they are not a one-size-fits-all solution. To determine if a business problem is suitable for an LLM solution, consider the following criteria:

## 1. Nature of the Problem
- **Text-Centric Tasks:** LLMs excel at tasks involving natural language such as:
  - Summarization
  - Translation
  - Text generation
  - Sentiment analysis
  - Question answering
  - Chatbots and conversational agents
- **Complex Reasoning or Structured Data:** LLMs may struggle with problems requiring precise numerical calculations, structured data processing, or domain-specific knowledge without fine-tuning.

## 2. Data Availability
- LLMs require large amounts of textual data (often unstructured) either for fine-tuning or prompt engineering.
- If your problem involves proprietary or sensitive data, consider data privacy and compliance issues.

## 3. Output Requirements
- **Flexible, Natural Language Outputs:** If your solution requires generating or interpreting human-like text, LLMs are suitable.
- **High Accuracy and Deterministic Outputs:** For tasks requiring exact answers (e.g., legal contracts, medical diagnosis), LLMs should be used cautiously or supplemented with domain-specific validation.

## 4. Integration Complexity
- Can the problem solution be integrated into existing workflows via APIs or applications that utilize LLMs?
- Are stakeholders comfortable with the probabilistic nature of LLM outputs?

## 5. Cost and Performance
- LLM inference can be costly and latency-sensitive.
- Evaluate if the business benefits justify the cost of deployment and maintenance.

## 6. Ethical and Compliance Considerations
- Consider bias, fairness, and ethical implications.
- Ensure compliance with regulations related to AI and data usage.

---

## Summary Checklist

| Criteria                     | Suitable if…                                      |
|------------------------------|-------------------------------------------------|
| Text-centric task             | Problem involves understanding/generating text  |
| Data availability             | Sufficient relevant textual data available       |
| Output nature                 | Flexible, natural language outputs acceptable    |
| Accuracy requirement          | Some tolerance for probabilistic answers         |
| Integration feasibility       | Can be integrated via APIs or apps                |
| Cost-benefit analysis         | Business value outweighs cost and complexity      |
| Ethical/compliance concerns   | Risks are manageable and compliant                 |

---

## Additional Tips
- Start with a **proof of concept** to evaluate effectiveness.
- Combine LLMs with other AI or rule-based systems for better accuracy.
- Use prompt engineering to tailor LLM outputs to your needs before considering fine-tuning.

---

If your business problem aligns well with these considerations, an LLM-based solution is worth exploring.
